## 1. 라이브러리 및 데이터 불러오기

In [1]:
# 분석에 필요한 라이브러리를 불러옵니다.
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
base_path = "C:/Users/gmlkd/data/시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/"
train_err = pd.read_csv(base_path+"train_err_data.csv")
train_problem = pd.read_csv(base_path+"train_problem_data.csv")
train_quality = pd.read_csv(base_path+"train_quality_data.csv")
test_err = pd.read_csv(base_path+"test_err_data.csv")
test_quality = pd.read_csv(base_path+"test_quality_data.csv")
submission = pd.read_csv(base_path+"sample_submission.csv")

C:\Users\gmlkd\AppData\Local\Temp\ipykernel_19424\2401973358.py:4: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  train_quality = pd.read_csv(base_path+"train_quality_data.csv")
C:\Users\gmlkd\AppData\Local\Temp\ipykernel_19424\2401973358.py:6: DtypeWarning: Columns (4,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  test_quality = pd.read_csv(base_path+"test_quality_data.csv")


In [3]:
display(train_err.head(2))
display(test_err.head(2))
display(train_quality.head(2))
display(test_quality.head(2))
display(train_problem.head(2))

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1


,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
1,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0


,user_id,time
0,19224,20201102200000
1,23664,20201116140000


In [4]:
train_err.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   int64 
 5   errcode   object
dtypes: int64(3), object(3)
memory usage: 757.8+ MB


In [5]:
train_problem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5429 entries, 0 to 5428
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  5429 non-null   int64
 1   time     5429 non-null   int64
dtypes: int64(2)
memory usage: 85.0 KB


In [6]:
train_quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        828624 non-null  int64  
 1   user_id     828624 non-null  int64  
 2   fwver       788544 non-null  object 
 3   quality_0   684192 non-null  float64
 4   quality_1   828624 non-null  int64  
 5   quality_2   788511 non-null  float64
 6   quality_3   828624 non-null  int64  
 7   quality_4   828624 non-null  int64  
 8   quality_5   828604 non-null  object 
 9   quality_6   828624 non-null  int64  
 10  quality_7   828624 non-null  object 
 11  quality_8   828624 non-null  object 
 12  quality_9   828624 non-null  object 
 13  quality_10  828624 non-null  object 
 14  quality_11  828624 non-null  int64  
 15  quality_12  828624 non-null  int64  
dtypes: float64(2), int64(8), object(6)
memory usage: 101.2+ MB


In [7]:
test_quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747972 entries, 0 to 747971
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        747972 non-null  int64  
 1   user_id     747972 non-null  int64  
 2   fwver       725208 non-null  object 
 3   quality_0   641388 non-null  float64
 4   quality_1   747961 non-null  object 
 5   quality_2   726857 non-null  float64
 6   quality_3   747972 non-null  int64  
 7   quality_4   747972 non-null  int64  
 8   quality_5   747928 non-null  object 
 9   quality_6   747972 non-null  int64  
 10  quality_7   747972 non-null  object 
 11  quality_8   747972 non-null  object 
 12  quality_9   747972 non-null  object 
 13  quality_10  747972 non-null  object 
 14  quality_11  747972 non-null  int64  
 15  quality_12  747972 non-null  int64  
dtypes: float64(2), int64(7), object(7)
memory usage: 91.3+ MB


## 2. 데이터 전처리

### 2-1. 결측치 처리

In [8]:
# time column을 datetype으로 변경

train_err['time'] = pd.to_datetime(train_err['time'], format='%Y%m%d%H%M%S')
train_problem['time'] = pd.to_datetime(train_problem['time'], format='%Y%m%d%H%M%S')
train_quality['time'] = pd.to_datetime(train_quality['time'], format='%Y%m%d%H%M%S')
test_err['time'] = pd.to_datetime(test_err['time'], format='%Y%m%d%H%M%S')
test_quality['time'] = pd.to_datetime(test_quality['time'], format='%Y%m%d%H%M%S')

#### train_err

In [9]:
train_err.isnull().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     1
dtype: int64

In [10]:
train_err[train_err.errcode.isnull()]

,user_id,time,model_nm,fwver,errtype,errcode
3825744,13639,2020-11-21 19:17:18,model_2,04.33.1261,5,NaN


In [11]:
# errcode 40013일 때와 user_id, time, model_nm, fwver, errtype모두 같다
train_err.iloc[3825742:3825747]

,user_id,time,model_nm,fwver,errtype,errcode
3825742,13639,2020-11-21 17:40:58,model_2,04.33.1261,31,1
3825743,13639,2020-11-21 17:41:10,model_2,04.33.1261,31,0
3825744,13639,2020-11-21 19:17:18,model_2,04.33.1261,5,NaN
3825745,13639,2020-11-21 19:17:18,model_2,04.33.1261,5,40013
3825746,13639,2020-11-21 22:09:19,model_2,04.33.1261,15,1


In [12]:
# 따라서 같은 errcode일 것임을 유추하여 40013값을 넣어준다.
train_err.errcode = train_err.errcode.fillna('40013')

In [13]:
train_err.isnull().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     0
dtype: int64

In [14]:
# train_err의 중복값 제거

print(train_err.shape)
train_err = train_err.drop_duplicates(subset=None, keep='first')
print(train_err.shape)

(16554663, 6)
(15368001, 6)


In [15]:
# model_nm, fwver, errcode encoding 필요

train_err.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15368001 entries, 0 to 16554662
Data columns (total 6 columns):
 #   Column    Dtype         
---  ------    -----         
 0   user_id   int64         
 1   time      datetime64[ns]
 2   model_nm  object        
 3   fwver     object        
 4   errtype   int64         
 5   errcode   object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 820.7+ MB


#### train_quality

In [16]:
train_quality.isnull().sum()

time               0
user_id            0
fwver          40080
quality_0     144432
quality_1          0
quality_2      40113
quality_3          0
quality_4          0
quality_5         20
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

In [17]:
# 1. fwver 결측치

# case1. fwver=non을 이상치로 판단하고 모두 지우기
# case2. train_err에서 해당 user_id의 fwver값을 가져와 채우기

In [18]:
# train_err에는 fwver가 업그레이드 된 경우가 많이 있다.
# 결측치 비율이 적고, 어떤 fwver에서 로그가 찍혔는지 알기 힘드므로 case1 선택

fwver_null_index = list(train_quality[train_quality.fwver.isnull()].index)

train_quality = train_quality.drop(fwver_null_index, axis=0)
train_quality.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788544 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        788544 non-null  datetime64[ns]
 1   user_id     788544 non-null  int64         
 2   fwver       788544 non-null  object        
 3   quality_0   684192 non-null  float64       
 4   quality_1   788544 non-null  int64         
 5   quality_2   788511 non-null  float64       
 6   quality_3   788544 non-null  int64         
 7   quality_4   788544 non-null  int64         
 8   quality_5   788524 non-null  object        
 9   quality_6   788544 non-null  int64         
 10  quality_7   788544 non-null  object        
 11  quality_8   788544 non-null  object        
 12  quality_9   788544 non-null  object        
 13  quality_10  788544 non-null  object        
 14  quality_11  788544 non-null  int64         
 15  quality_12  788544 non-null  int64         
dtypes:

In [19]:
# 2. quality 결측치

# case1. 최빈값 채우기
# case2. 평균값 채우기

# quality_0
train_quality.quality_0.describe()

count    684192.000000
mean          4.148701
std         479.315029
min          -1.000000
25%           0.000000
50%           0.000000
75%           0.000000
max      157667.000000
Name: quality_0, dtype: float64

In [20]:
train_quality.quality_0.value_counts()

 0.0       542790
-1.0       130828
 1.0         2097
 2.0         1252
 3.0          518
            ...  
 1113.0         1
 1177.0         1
 1186.0         1
 946.0          1
 2351.0         1
Name: quality_0, Length: 753, dtype: int64

In [21]:
# quality_2
train_quality.quality_2.describe()

count    788511.000000
mean          4.751094
std         586.252469
min          -1.000000
25%           0.000000
50%           0.000000
75%           0.000000
max      191859.000000
Name: quality_2, dtype: float64

In [22]:
train_quality.quality_2.value_counts()

 0.0        632469
-1.0        144392
 1.0          2937
 2.0          1073
 3.0           580
             ...  
 872.0           1
 11259.0         1
 373.0           1
 8278.0          1
 2351.0          1
Name: quality_2, Length: 798, dtype: int64

In [23]:
# quality_5
train_quality.quality_5.value_counts()

# -1, 0 ,1에 데이터가 편향되어있고, max값과 차이가 많이 나므로 평균보단 최빈값으로?

0         402947
-1        144391
1          55206
2          35399
3          21118
           ...  
10,490         1
10,821         1
12,379         1
14,523         1
9,558          1
Name: quality_5, Length: 4744, dtype: int64

In [24]:
# case1. 최빈값으로 채우기

for i in train_quality.columns[3:]:
    train_quality[i] = train_quality[i].fillna(train_quality[i].mode()[0])

In [25]:
train_quality.isnull().sum()

time          0
user_id       0
fwver         0
quality_0     0
quality_1     0
quality_2     0
quality_3     0
quality_4     0
quality_5     0
quality_6     0
quality_7     0
quality_8     0
quality_9     0
quality_10    0
quality_11    0
quality_12    0
dtype: int64

In [26]:
# train_quality 중복값 제거

print(train_quality.shape)
train_quality = train_quality.drop_duplicates(subset=None, keep='first')
print(train_quality.shape)

(788544, 16)
(272452, 16)


#### test_err

In [27]:
test_err.isnull().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     4
dtype: int64

In [28]:
test_err[test_err.errcode.isnull()] 

,user_id,time,model_nm,fwver,errtype,errcode
937967,30820,2020-11-15 04:43:17,model_2,04.33.1261,5,NaN
4038892,33681,2020-11-03 11:02:59,model_2,04.33.1185,5,NaN
9486881,38991,2020-11-27 21:38:38,model_2,04.33.1261,5,NaN
10425473,39894,2020-11-28 14:47:12,model_1,04.16.3553,5,NaN


In [29]:
# 40053으로 채우기
test_err.iloc[937965:937970]

,user_id,time,model_nm,fwver,errtype,errcode
937965,30820,2020-11-15 03:59:00,model_2,04.33.1261,40,1
937966,30820,2020-11-15 03:59:02,model_2,04.33.1261,40,0
937967,30820,2020-11-15 04:43:17,model_2,04.33.1261,5,NaN
937968,30820,2020-11-15 04:43:17,model_2,04.33.1261,5,40053
937969,30820,2020-11-15 09:10:24,model_2,04.33.1261,15,1


In [30]:
# 40053으로 채우기
test_err.iloc[4038890:4038895]

,user_id,time,model_nm,fwver,errtype,errcode
4038890,33681,2020-11-03 11:01:47,model_2,04.33.1185,14,14
4038891,33681,2020-11-03 11:02:59,model_2,04.33.1185,7,14
4038892,33681,2020-11-03 11:02:59,model_2,04.33.1185,5,NaN
4038893,33681,2020-11-03 11:02:59,model_2,04.33.1185,5,40053
4038894,33681,2020-11-03 11:03:00,model_2,04.33.1185,6,14


In [31]:
# 40053으로 채우기
test_err.iloc[9486879:9486883]

,user_id,time,model_nm,fwver,errtype,errcode
9486879,38991,2020-11-27 18:59:52,model_2,04.33.1261,5,B-A8002
9486880,38991,2020-11-27 21:38:38,model_2,04.33.1261,5,40053
9486881,38991,2020-11-27 21:38:38,model_2,04.33.1261,5,NaN
9486882,38991,2020-11-28 00:13:46,model_2,04.33.1261,26,1


In [32]:
# -1010으로 채우기
test_err.iloc[10425470:10425475]

,user_id,time,model_nm,fwver,errtype,errcode
10425470,39894,2020-11-28 14:46:05,model_1,04.16.3553,20,1
10425471,39894,2020-11-28 14:46:21,model_1,04.16.3553,26,1
10425472,39894,2020-11-28 14:47:12,model_1,04.16.3553,5,-1010
10425473,39894,2020-11-28 14:47:12,model_1,04.16.3553,5,NaN
10425474,39894,2020-11-28 14:47:57,model_1,04.16.3553,32,80


In [33]:
# 결측치 채우기
test_err.iloc[937967, 5] = '40053'
test_err.iloc[4038892, 5] = '40053'
test_err.iloc[9486881, 5] = '40053'
test_err.iloc[10425473, 5] = '-1010'

In [34]:
test_err.isnull().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     0
dtype: int64

In [35]:
# test_err 중복값 제거

print(test_err.shape)
test_err = test_err.drop_duplicates(subset=None, keep='first')
print(test_err.shape)

(16532648, 6)
(15527221, 6)


#### test_quality

In [36]:
test_quality.isnull().sum()

time               0
user_id            0
fwver          22764
quality_0     106584
quality_1         11
quality_2      21115
quality_3          0
quality_4          0
quality_5         44
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

In [37]:
# fwver결측치 제거

fwver_null_index = list(test_quality[test_quality.fwver.isnull()].index)

test_quality = test_quality.drop(fwver_null_index, axis=0)
test_quality.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 725208 entries, 0 to 747971
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        725208 non-null  datetime64[ns]
 1   user_id     725208 non-null  int64         
 2   fwver       725208 non-null  object        
 3   quality_0   639780 non-null  float64       
 4   quality_1   725197 non-null  object        
 5   quality_2   725177 non-null  float64       
 6   quality_3   725208 non-null  int64         
 7   quality_4   725208 non-null  int64         
 8   quality_5   725164 non-null  object        
 9   quality_6   725208 non-null  int64         
 10  quality_7   725208 non-null  object        
 11  quality_8   725208 non-null  object        
 12  quality_9   725208 non-null  object        
 13  quality_10  725208 non-null  object        
 14  quality_11  725208 non-null  int64         
 15  quality_12  725208 non-null  int64         
dtypes:

In [38]:
# quality_0 ~ 12 최빈값으로 채우기

for i in test_quality.columns[3:]:
    test_quality[i] = test_quality[i].fillna(test_quality[i].mode()[0])

In [39]:
test_quality.isnull().sum()

time          0
user_id       0
fwver         0
quality_0     0
quality_1     0
quality_2     0
quality_3     0
quality_4     0
quality_5     0
quality_6     0
quality_7     0
quality_8     0
quality_9     0
quality_10    0
quality_11    0
quality_12    0
dtype: int64

In [40]:
# test_quality 중복값 제거

print(test_quality.shape)
test_quality = test_quality.drop_duplicates(subset=None, keep='first')
print(test_quality.shape)

(725208, 16)
(237629, 16)


In [41]:
# train/test_quality에서 쉼표로 인해 string형이 된  data 변경

def strToInt(x):
    if type(x) == str:
        x = x.replace(",", "")
        x = int(x)
        return x
    else:
        x = int(x)
        return x

In [42]:
# train_quality
for i in train_quality.columns[3:]:
    train_quality[i] = train_quality[i].apply(lambda x: strToInt(x))
    
# test_quality
for i in test_quality.columns[3:]:
    test_quality[i] = test_quality[i].apply(lambda x: strToInt(x))

In [43]:
train_quality.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272452 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        272452 non-null  datetime64[ns]
 1   user_id     272452 non-null  int64         
 2   fwver       272452 non-null  object        
 3   quality_0   272452 non-null  int64         
 4   quality_1   272452 non-null  int64         
 5   quality_2   272452 non-null  int64         
 6   quality_3   272452 non-null  int64         
 7   quality_4   272452 non-null  int64         
 8   quality_5   272452 non-null  int64         
 9   quality_6   272452 non-null  int64         
 10  quality_7   272452 non-null  int64         
 11  quality_8   272452 non-null  int64         
 12  quality_9   272452 non-null  int64         
 13  quality_10  272452 non-null  int64         
 14  quality_11  272452 non-null  int64         
 15  quality_12  272452 non-null  int64         
dtypes:

In [44]:
test_quality.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237629 entries, 0 to 747967
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        237629 non-null  datetime64[ns]
 1   user_id     237629 non-null  int64         
 2   fwver       237629 non-null  object        
 3   quality_0   237629 non-null  int64         
 4   quality_1   237629 non-null  int64         
 5   quality_2   237629 non-null  int64         
 6   quality_3   237629 non-null  int64         
 7   quality_4   237629 non-null  int64         
 8   quality_5   237629 non-null  int64         
 9   quality_6   237629 non-null  int64         
 10  quality_7   237629 non-null  int64         
 11  quality_8   237629 non-null  int64         
 12  quality_9   237629 non-null  int64         
 13  quality_10  237629 non-null  int64         
 14  quality_11  237629 non-null  int64         
 15  quality_12  237629 non-null  int64         
dtypes:

### 2-2. 상관관계 분석

In [45]:
# 피처간 비슷한 변수들이 있는지 확인하기 위해 상관관계 분석을 수행합니다.


In [46]:
# heatmap


### 2-3. 이상치 검출

1) IQR(Inter-Quantile Range)


2) Outlier Detection method (e.g. Isolation Forest)

## 3. 예측 모델 구현

In [47]:
# 모델을 불러옵니다


In [48]:
# 학습에 필요한 데이터셋을 만듭니다


In [49]:
# 학습 및 모델 검증을 위해 KFold Cross Validation 기법을 사용합니다.


In [50]:
# training

In [51]:
# submission 파일 생성